In [ ]:
data = pd.read_csv("btc_tx_2011_2013.csv") #13- 14 - 15 e bakalim corr var mi hangi featurelar arasi average shortest path length etc.
# flow bazli metrikler
# her biri icin graph olusturulup time seriese bagli graphlar
# makalelerdeki featurelar (stanford 2015 | Group work) aynilarini deneriz
# 100k nodes randomly sampled, multiple times. Per month and closure price, price changing return % 
# pearson spearman corrs

In [ ]:
data.head()

In [ ]:
data.columns = ["tx_hash_from","tx_hash_to","datetime","amount_bitcoins"]

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data["datetime"]=pd.to_datetime(data["datetime"])
data["datetime"] = data["datetime"].dt.date

In [ ]:
data

In [ ]:
df = data.sort_values(by= ["datetime"])

In [ ]:
df.to_csv('Sorted.csv')

In [ ]:
df = pd.read_csv("Sorted.csv")

In [ ]:
df["datetime"]=pd.to_datetime(df["datetime"])
df["datetime"] = df["datetime"].dt.date

In [ ]:
sliced = slice_by_weeks(df)

In [ ]:
with open('slices.pkl', 'wb') as f:
    pickle.dump(sliced, f)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from datetime import datetime
import gc
import pickle

In [2]:
def slice_by_weeks(df):
    groups = []
    prev=0
    end_of_week1 = False
    end_of_week2 = False
    for i in range(df.shape[0]):
        if df["datetime"].iloc[i].weekday() == 6:
            end_of_week1 = True
        elif df["datetime"].iloc[i].weekday() == 0:
            end_of_week2 = True
        if end_of_week1 and end_of_week2:
            #print(prev, i)
            groups.append(df.iloc[prev:i])
            prev = i
            end_of_week1 = False
            end_of_week2 = False
    return groups

def createGraph(data): # directed graph
    G=nx.from_pandas_edgelist(data,source="tx_hash_from",target ="tx_hash_to", edge_attr="amount_bitcoins", create_using=nx.DiGraph())
    return G

In [3]:
with open('slices.pkl', 'rb') as f:
    sliced = pickle.load(f)

In [4]:
len(sliced)

519

In [5]:
graphs = []
index =470
for i in sliced[470:519]:
    graphs.append(createGraph(i))
    index+=1
    print(index, end="\r")

In [6]:
with open('graphs470-519.pkl', 'wb') as f:
    pickle.dump(graphs, f)